In [1]:
import torch 
import torch.nn as nn
import numpy as np
torch.__version__

'2.9.0+cpu'

### 3.1 logistic回归实战

#### logistic回归介绍
logistic回归是一种广义线性回归（generalized linear model），与多重线性回归分析有很多相同之处。它们的模型形式基本上相同，都具有 wx + b，其中w和b是待求参数，其区别在于他们的因变量不同，多重线性回归直接将wx+b作为因变量，即y =wx+b,而logistic回归则通过函数L将wx+b对应一个隐状态p，p =L(wx+b),然后根据p 与1-p的大小决定因变量的值。如果L是logistic函数，就是logistic回归，如果L是多项式函数就是多项式回归。

说的更通俗一点，就是logistic回归会在线性回归后再加一层logistic函数的调用。

logistic回归主要是进行二分类预测，我们在激活函数时候讲到过 Sigmod函数，Sigmod函数是最常见的logistic函数，因为Sigmod函数的输出的是是对于0~1之间的概率值，当概率大于0.5预测为1，小于0.5预测为0。

#### UCI German Credit 数据集
UCI German Credit是UCI的德国信用数据集，里面有原数据和数值化后的数据。

German Credit数据是根据个人的银行贷款信息和申请客户贷款逾期发生情况来预测贷款违约倾向的数据集，数据集包含24个维度的，1000条数据，在这里我们直接使用处理好的数值化的数据，作为展示。

### 3.2 代码实战
我们这里使用的 german.data-numeric是numpy处理好数值化数据，我们直接使用numpy的load方法读取即可

In [2]:
data = np.loadtxt("german.data-numeric")

FileNotFoundError: german.data-numeric not found.

数据读取完成后我们要对数据做一下归一化的处理

In [3]:
n, l = data.shape
for i in range(l-1):
    meanVal = np.mean(data[:, i])
    stdVal = np.std(data[:, i])
    data[:, i] = (data[:, i] - meanVal) / stdVal

NameError: name 'data' is not defined

打乱数据

In [4]:
np.random.shuffle(data)

NameError: name 'data' is not defined

区分训练集与测试集，由于这里没有验证集，所以我们直接使用测试集的准确度作为评判好坏的标准。

区分规则：900条用于训练，100条作为测试。

`german.data-numeric`的格式为，前24列为24个维度，最后一个为要打的标签（0，1），所以我们将数据和标签一起区分出来

In [5]:
train_data = data[:900, : l-1]
train_lab = data[:900, l-1] -1 # 标签从0开始
test_data = data[900:, : l-1]
test_lab = data[900:, l-1] -1

NameError: name 'data' is not defined

In [7]:
# 定义模型
class LR(nn.Module):
    def __init__(self):
        super(LR, self).__init__()
        self.fc = nn.Linear(24, 2)  # 24个输入，2个输出

    def forward(self, x):
        out = self.fc(x)
        out = torch.sigmoid(out)
        return out

    # 测试集上的准确率
    def test(pred, lab):
        t = pred.max(-1)[1] == lab # 取最大值的索引作为预测类别
        return torch.mean(t.float())

In [8]:
net = LR()
criterion = nn.CrossEntropyLoss() # 交叉熵损失函数
optm = torch.optim.Adam(net.parameters())  # Adam优化器
epochs = 1000

In [ ]:
for i in range(epochs):
    # 指定模型为训练模型
    net.train()
    # 输入值都需要转化为torch的tensor格式
    x = torch.form_numpy(train_data).float()
    y = torch.from_numpy(train_lab).long() # 标签为整数格式
    y_hat = net(x)
    loss = criterion(y_hat, y)
    optm.zero_grad()
    loss.backward()
    optm.step() # 更新参数

    if (i+1) % 100 == 0: # 每100次输出一次结果
        net.eval()  # 指定模型为评估模式
        test_in = torch.from_numpy(test_data).float()
        test_l = torch.form_numpy(test_lab).long()
        test_out = net(test_in)

        # 计算测试集上的准确率
        accu = test(test_out, test_l)
        print("Epoch:{}, Loss:{: 4f}, Accuracy:{: 2f}".format(i+1, loss.item(), accu))